In [4]:
import cPickle as pickle
import sys

sys.path.append('../')

from preprocessing.trainingdata import TrainingData

In [2]:
def load(filename):
    return pickle.load(open(filename, 'rb'))

In [7]:
audio = load('../data/MUS/MAPS_MUS-alb_se3_AkPnBsdf.pkl')